In [1]:
!pip install newsapi-python
!pip install alpaca-trade-api
!pip install websockets>=13.0.0,<15.1.0
!pip install langchain langchain-google-genai python-dotenv
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.3 MB/s eta 0:00:00
  Created wheel for msgpack: filename=msgpack-1.0.3-cp311-cp311-linux_x86_64.whl size=15688 sha256=85f07a572ecc7b301d04127cddb8550b610fb66cd1e50596aaab2b853dc1c5ab
  Stored in directory: /root/.cache/pip/wheels/f6/35/da/ed9b26b510235e00e3a3c3bab7bad97b59214729662255ab3d
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.1.1
    Uninstalling msgpack-1.1.1:
      Successfully uninstalled msgpack-1.1.1
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 10.4
    Uninstalling websockets-10.4:
      Successfully uninstalled websockets-10.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
alpaca-trade-api 3.2.0 requires websockets<11,>=9.0, but you have websockets 15.0.1 which is incompatible.


In [5]:
import yfinance as yf
import pandas as pd
from newsapi import NewsApiClient
import datetime as dt
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [6]:
from langchain.agents import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub

In [20]:
import alpaca_trade_api as tradeapi

In [ ]:
NEWS_API_KEY = None
OPENAI_API_KEY = None
GEMINI_API_KEY = None
ALPACA_API_KEY = None
ALPACA_SECRET_KEY = None

In [8]:
@tool
def get_stock_data(ticker_symbol: str) -> str:
    """
    Use this tool to get the latest stock price data for a company.
    The input should be the company's ticker symbol (e.g., AAPL, GOOGL).
    """
    # Create a Ticker object for the stock
    stock = yf.Ticker(ticker_symbol)

    # Get historical market data
    # Common periods are "1d", "5d", "1mo", "3mo", "6mo", "1y", "5y", "max"
    hist_data = stock.history(period="1mo")

    print(f"--- Historical Stock Prices for {ticker_symbol} ---")
    print(hist_data.head()) # .head() shows the first 5 rows

    return hist_data

In [9]:
@tool
def get_news_articles(query: str) -> str:
    """
    Use this tool to get recent news articles for a company or topic.
    The input should be the company name or a search query (e.g., Apple Inc, Tesla electric vehicles).
    """
    # Initialize the News API client
    newsapi = NewsApiClient(api_key=NEWS_API_KEY)

    # Calculate yesterday's date for the query
    yesterday = (dt.date.today() - dt.timedelta(days=1)).strftime('%Y-%m-%d')

    # Fetch the news
    # We search for the query in English, sorted by relevancy
    all_articles = newsapi.get_everything(
        q=query,
        language='en',
        sort_by='relevancy',
        from_param=yesterday # Get news from the last 24 hours
    )

    print(f"\n--- Recent News for {query} ---")
    # Print the titles of the first 5 articles
    for article in all_articles['articles'][:5]:
        print(f"Title: {article['title']}")
        print(f"Source: {article['source']['name']}")
        print("-" * 100)

    return all_articles['articles']




In [10]:
@tool
def analyze_sentiment(text: str) -> str:
    """
    Use this tool to analyze the sentiment of a piece of text (like news headlines).
    The input should be the text you want to analyze.
    """
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    prediction_probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

    positive = prediction_probs[0][0].item()
    negative = prediction_probs[0][1].item()
    neutral = prediction_probs[0][2].item()

    return f"Sentiment Analysis: Positive: {positive:.2f}, Negative: {negative:.2f}, Neutral: {neutral:.2f}"

In [11]:
tools = [get_stock_data, get_news_articles, analyze_sentiment]

In [12]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=GEMINI_API_KEY, temperature=0)

In [13]:
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
agent = create_react_agent(llm, tools, prompt)

In [15]:
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, verbose=True)

In [35]:
ticker = "NVDA"
company_name = "NVIDIA Corp"

report_template = f"""
    Generate a stock analysis report for {company_name} ({ticker}).
    Your final answer MUST be structured in the following format. Use the tools to find the information for each section:

    # Stock Analysis Report: {company_name} ({ticker})
    **Date:** {dt.date.today().strftime('%Y-%m-%d')}

    ## 1. Overall Recommendation
    (Your single-word recommendation: **STRONG BUY**, **MONITOR**, or **AVOID**. Base this on the news and sentiment.)

    ## 2. Key Price Metrics
    (Use the get_stock_data tool to report the last closing price and briefly describe the recent 5-day trend.)

    ## 3. News & Sentiment Summary
    (Use get_news_articles and analyze_sentiment to create a 2-3 sentence summary of the key news themes and the overall sentiment.)

    ## 4. Top Headlines
    (List the top 3-4 most relevant news headlines you found using the get_news_articles tool.)

    ---
    *Disclaimer: This is an AI-generated report for informational purposes only and does not constitute financial advice.*
    """

print(f"--- Generating Report for {company_name} ---")

response = agent_executor.invoke({"input": report_template})

--- Generating Report for NVIDIA Corp ---


> Entering new AgentExecutor chain...
Action: get_stock_data
Action Input: NVDA--- Historical Stock Prices for NVDA ---
                                 Open        High         Low       Close  \
Date                                                                        
2025-07-09 00:00:00-04:00  161.220001  164.419998  161.160004  162.880005   
2025-07-10 00:00:00-04:00  164.320007  164.500000  161.610001  164.100006   
2025-07-11 00:00:00-04:00  163.720001  167.889999  163.470001  164.919998   
2025-07-14 00:00:00-04:00  165.369995  165.490005  162.020004  164.070007   
2025-07-15 00:00:00-04:00  171.190002  172.399994  169.199997  170.699997   

                              Volume  Dividends  Stock Splits  
Date                                                           
2025-07-09 00:00:00-04:00  183656400        0.0           0.0  
2025-07-10 00:00:00-04:00  167704100        0.0           0.0  
2025-07-11 00:00:00-04:00  193633300    

In [36]:
print("\n--- Agent's Final Answer ---")
final_report = response['output']
print(final_report)


--- Agent's Final Answer ---
# Stock Analysis Report: NVIDIA Corp (NVDA)
**Date:** 2025-08-10

## 1. Overall Recommendation
**STRONG BUY**

## 2. Key Price Metrics
NVIDIA Corp's last closing price as of 2025-08-08 was $182.70. Over the recent 5-day period (from 2025-08-04 to 2025-08-08), the stock has shown a generally positive trend, increasing from $180.00 to $182.70, indicating upward momentum.

## 3. News & Sentiment Summary
Recent news indicates a highly positive development for NVIDIA, as the U.S. government has begun granting licenses for the company to export its H20 chips to China. This move removes a significant trade hurdle and is expected to avert potential losses, allowing NVIDIA to access a key market. The overall sentiment surrounding these developments is strongly positive.

## 4. Top Headlines
*   US allowed Nvidia chip sales after Trump-Huang meeting
*   Nvidia wins US approval to sell H20 chips to China
*   US licenses Nvidia to export chips to China, official says


In [ ]:
def execute_paper_trade(ticker, qty=1, side='buy'):
    """
    Executes a paper trade on Alpaca.
    """
    api = tradeapi.REST(ALPACA_API_KEY,
        ALPACA_SECRET_KEY,
        base_url='https://paper-api.alpaca.markets',
        api_version='v2'
    )

    try:
        print(f"--- Attempting to {side} {qty} share(s) of {ticker} ---")
        order = api.submit_order(
            symbol=ticker,
            qty=qty,
            side=side,
            type='market',
            time_in_force='day'
        )
        print(f"Success! Order submitted:")
        print(order)
        return order
    except Exception as e:
        print(f"Error executing trade: {e}")
        return None

In [ ]:
def get_position(ticker):
    api = tradeapi.REST(ALPACA_API_KEY,
        ALPACA_SECRET_KEY,
        base_url='https://paper-api.alpaca.markets',
        api_version='v2'
    )

    try:
        position = api.get_position(ticker)
        qty = float(position.qty)
        print(f" Found existing position: {qty} share(s) of {ticker}.")
        return qty
    except tradeapi.rest.APIError as e:
        if e.status_code == 404:
            print(f"No existing position found for {ticker}.")
            return 0
        else:
            print(f" Error getting position: {e}")
            return 0

In [45]:
if "Overall Recommendation" in final_report:
    # --- BUY LOGIC WITH CONFIRMATION ---
    if "**STRONG BUY**" in final_report:
        print("\nRecommendation is 'STRONG BUY'.")
        confirmation = input("Do you want to execute a BUY trade? (y/n): ").lower()

        if confirmation == 'y':
            while True: # Loop for input validation
                try:
                    qty_to_buy = int(input("Enter quantity to buy: "))
                    if qty_to_buy > 0:
                        execute_paper_trade(ticker, qty=qty_to_buy, side='buy')
                        break # Exit loop on success
                    else:
                        print("Quantity must be a positive number.")
                except ValueError:
                    print("Invalid input. Please enter a whole number for the quantity.")
        else:
            print("Trade aborted by user.")

    # --- SELL LOGIC WITH CONFIRMATION ---
    elif "**AVOID**" in final_report:
        print("\nRecommendation is 'AVOID'.")
        owned_qty = get_position(ticker)

        if owned_qty > 0:
            confirmation = input(f"You own {owned_qty} share(s). Do you want to execute a SELL trade? (y/n): ").lower()

            if confirmation == 'y':
                while True: # Loop for input validation
                    try:
                        qty_to_sell = int(input(f"Enter quantity to sell (up to {owned_qty}): "))
                        if 0 < qty_to_sell <= owned_qty:
                            execute_paper_trade(ticker, qty=qty_to_sell, side='sell')
                            break # Exit loop on success
                        else:
                            print(f"Invalid quantity. Please enter a number between 1 and {int(owned_qty)}.")
                    except ValueError:
                        print("Invalid input. Please enter a whole number for the quantity.")
            else:
                print("Trade aborted by user.")
        else:
            print("No existing position to sell.")

    else:
        print(f"\nRecommendation is 'MONITOR'. No trade executed.")
else:
    print("\nCould not find 'Overall Recommendation' in the report. No trade executed.")


Recommendation is 'STRONG BUY'.
Do you want to execute a BUY trade? (y/n): n
Trade aborted by user.


In [ ]:
# USE THIS IF THE ABOVE CODE DON'T WORK

# ---------------------------------------------------------------------------------------------------------------
# if "Overall Recommendation" in final_report:
#     if "**STRONG BUY**" in final_report:
#         print("\nRecommendation is 'STRONG BUY'. Executing paper trade.")
#         execute_paper_trade(ticker, qty=5, side='buy')
#     else:
#         print(f"\nRecommendation found, but it was not 'STRONG BUY'. No trade executed.")
# else:
#     print("\nCould not find 'Overall Recommendation' in the report. No trade executed.")
# ---------------------------------------------------------------------------------------------------------------